In [0]:
# https://keras.io/
#!pip install -q keras
#import keras

In [0]:
#import keras
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from tensorflow.keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
#k.tensorflow_backend.set_session(tf.Session(config=config))
k.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

### Using TFRecords from deep-falcon library (https://github.com/ksasi/deep-falcon)

In [5]:
# Import TFRecords module from deep-falcon library from Github (https://github.com/ksasi/deep-falcon)

!git clone https://github.com/ksasi/deep-falcon ./deep-falcon                                                   

import sys
sys.path.append("/content/deep-falcon/")

Cloning into './deep-falcon'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 36 (delta 14), reused 25 (delta 6), pack-reused 0
Unpacking objects: 100% (36/36), done.


In [6]:
from falcon import data

# Load CIFAR10 Data as TFRecords
names = data.gen_tfrecords("CIFAR10")
print(names)

W0901 15:09:28.782018 140501388572544 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0901 15:09:28.785160 140501388572544 deprecation.py:323] From /content/deep-falcon/falcon/data.py:41: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0901 15:09:28.787071 140501388572544 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is depr

Successfully downloaded cifar-10-python.tar.gz 170498071 bytes.


W0901 15:09:33.412395 140501388572544 deprecation_wrapper.py:119] From /content/deep-falcon/falcon/data.py:82: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W0901 15:09:33.414250 140501388572544 deprecation_wrapper.py:119] From /content/deep-falcon/falcon/data.py:71: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.



Generating ./tfrecords_data/train.tfrecords
Generating ./tfrecords_data/test.tfrecords
['./tfrecords_data/train.tfrecords', './tfrecords_data/test.tfrecords']


In [0]:
test_filename = names[0]
train_filename = names[1]

ds_test = tf.data.TFRecordDataset(test_filename).map(data.parser, num_parallel_calls=2).shuffle(buffer_size=50000).batch(batch_size).repeat()

ds_train = tf.data.TFRecordDataset(train_filename).map(data.parser, num_parallel_calls=2).shuffle(buffer_size=10000).batch(batch_size).repeat()


In [9]:
print(ds_test)

<DatasetV1Adapter shapes: ((?, 32, 32, 3), (?, 10)), types: (tf.float32, tf.float32)>


In [10]:

'''(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)'''

'(x_train, y_train), (x_test, y_test) = cifar10.load_data()\nimg_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]\n\n# convert to one hot encoing \ny_train = keras.utils.to_categorical(y_train, num_classes)\ny_test = keras.utils.to_categorical(y_test, num_classes)'

In [0]:
img_height = 32
img_width = 32
channel = 3

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        #Conv2D_3_3 = Conv2D(int(num_filter), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [20]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_52 (Conv2D)              (None, 32, 32, 12)   324         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_52 (BatchNo (None, 32, 32, 12)   48          conv2d_52[0][0]                  
__________________________________________________________________________________________________
activation_52 (Activation)      (None, 32, 32, 12)   0           batch_normalization_52[0][0]     
______________________________________________________________________________________________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [22]:
import numpy as np

steps_per_epoch = np.ceil(50000/batch_size)
validation_steps = np.ceil(10000/batch_size)

model.fit(ds_train,steps_per_epoch=steps_per_epoch,epochs=epochs,verbose=1,validation_data=ds_test, validation_steps=validation_steps)

Epoch 1/50
391/391 [==============================] - 118s 302ms/step - loss: 1.7052 - acc: 0.3634 - val_loss: 1.5538 - val_acc: 0.4221
Epoch 2/50
391/391 [==============================] - 107s 274ms/step - loss: 1.3759 - acc: 0.4910 - val_loss: 1.4962 - val_acc: 0.4743
Epoch 3/50
391/391 [==============================] - 107s 274ms/step - loss: 1.2178 - acc: 0.5540 - val_loss: 1.4691 - val_acc: 0.4952
Epoch 4/50
391/391 [==============================] - 107s 274ms/step - loss: 1.1111 - acc: 0.5950 - val_loss: 1.2704 - val_acc: 0.5758
Epoch 5/50
391/391 [==============================] - 107s 275ms/step - loss: 1.0301 - acc: 0.6270 - val_loss: 1.3183 - val_acc: 0.5621
Epoch 6/50
391/391 [==============================] - 107s 275ms/step - loss: 0.9526 - acc: 0.6578 - val_loss: 1.2190 - val_acc: 0.6027
Epoch 7/50
391/391 [==============================] - 108s 276ms/step - loss: 0.8980 - acc: 0.6783 - val_loss: 1.2155 - val_acc: 0.6024
Epoch 8/50
391/391 [============================

In [24]:
# Test the model
score = model.evaluate(ds_test, steps = validation_steps, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

79/79 [==============================] - 12s 147ms/step - loss: 1.6074 - acc: 0.6751
Test loss: 1.6074248793758923
Test accuracy: 0.6751385


In [25]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')

### **References & Attributions:**

*   Keras (https://keras.io/)

*   Tensorflow 2.1 (https://www.tensorflow.org/api_docs/python/)


*Disclaimer: The contents of this notebook are used for educational purposes i.e. for learning and research.*